In [1]:
from pathlib import Path
wd = Path.cwd()
wd = wd / 'merged_data'
sub1 = str(wd) + '/subtask1.parquet'
sub2 = str(wd) + '/subtask1_test.parquet'
print(sub1)
print(sub2)

/home/mkukic/projektR2025/merged_data/subtask1.parquet
/home/mkukic/projektR2025/merged_data/subtask1_test.parquet


In [2]:
import pandas as pd
df = pd.read_parquet(sub1)
df2 = pd.read_parquet(sub2)
#print(df2.tail())
#print(df.tail())

In [3]:
df = pd.concat([df, df2], ignore_index=True)
#df = df[:100]
#df.tail()

In [4]:
import re
def labelNum(row):
    if row['class1'] == 'Antagonist':
        return int(0)
    if row['class1'] == 'Innocent':
        return int(1)
    if row['class1'] == 'Protagonist':
        return int(2)
def cleanText(row):
    text = str(row['text'])
    #text = re.sub(r'[^\w\s]', ' ', text)
    text = text.replace('\n',' ').replace('  ', ' ')
    return text
df['label1'] = df.apply(labelNum,axis=1)
df['input'] = df.apply(cleanText,axis=1)

In [5]:
def labelNum2(row):
    labels2 = [0 for _ in range(22)]
    if row['label1'] == 2:
        #labels2 = [0 for _ in range(6)]
        if 'Guardian' in row['classes2']:
            labels2[0] = 1
        if 'Martyr' in row['classes2']:
            labels2[1] = 1
        if 'Peacemaker' in row['classes2']:
            labels2[2] = 1
        if 'Rebel' in row['classes2']:
            labels2[3] = 1
        if 'Underdog' in row['classes2']:
            labels2[4] = 1
        if 'Virtuous' in row['classes2']:
            labels2[5] = 1
    elif row['label1'] == 0:
        #labels2 = [0 for _ in range(12)]
        if 'Instigator' in row['classes2']:
           labels2[6] = 1
        if 'Conspirator' in row['classes2']:
            labels2[7] = 1
        if 'Tyrant' in row['classes2']:
            labels2[8] = 1
        if  'Foreign Adversary' in row['classes2']:
            labels2[9] = 1
        if 'Traitor' in row['classes2']:
            labels2[10] = 1
        if 'Spy' in row['classes2']:
            labels2[11] = 1
        if 'Saboteur' in row['classes2']:
            labels2[12] = 1
        if 'Corrupt' in row['classes2']:
            labels2[13] = 1
        if 'Incompetent' in row['classes2']:
            labels2[14] = 1
        if 'Terrorist' in row['classes2']:
            labels2[15] = 1
        if 'Deceiver' in row['classes2']:
            labels2[16] = 1
        if 'Bigot' in row['classes2']:
            labels2[17] = 1
    elif row['label1'] == 1:
        #labels2 = [0 for _ in range(4)]
        if 'Forgotten' in row['classes2']:
            labels2[18] = 1
        if 'Exploited' in row['classes2']:
            labels2[19] = 1
        if 'Victim' in row['classes2']:
            labels2[20] = 1
        if 'Scapegoat' in row['classes2']:
            labels2[21] = 1
    return labels2

df['label2'] = df.apply(labelNum2, axis=1)

In [6]:
data = df.loc[ : , ['input','label2']]
data.aggregate

from sklearn.model_selection import train_test_split
import numpy as np
X = data["input"]
y = data["label2"]
lang = df['lang'].tolist()
lang = np.array(lang)


In [7]:
# for the confusion matrix in the end
from sklearn.metrics import precision_recall_fscore_support
from torch.optim import AdamW
final_preds = np.empty((0, 12), dtype=np.int8)
final_labels = np.empty((0, 12), dtype=np.int8)
#span_start_token_id = tokenizer.convert_tokens_to_ids('[SPAN_START]')
#span_end_token_id = tokenizer.convert_tokens_to_ids('[SPAN_END]')

In [8]:
from transformers import XLMRobertaForSequenceClassification, XLMRobertaTokenizerFast

#model = XLMRobertaForSequenceClassification.from_pretrained("xlm-roberta-base", num_labels=22,problem_type="multi_label_classification").to(device)
tokenizer = XLMRobertaTokenizerFast.from_pretrained("xlm-roberta-base")

/home/mkukic/.conda/envs/projektR2025/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
def preprocess_function(examples):
    return tokenizer(examples['text'], padding=True, truncation=True, max_length=512)

In [10]:
accuracies_model = []
accuracies_baseline =[]

In [11]:
import torch
from datasets import Dataset
from torch.utils.data import DataLoader
from tqdm import tqdm
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
def set_seed_everywhere(seed):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)

    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

Using device: cuda


In [12]:
seeds = [1,11,42,199,33,85,102]
import random
import os
#os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
for seed in seeds:
    print(f"Testing seed: {seed}")
    set_seed_everywhere(seed)

    X_train, X_test, y_train, y_test,lang_train,lang_test = train_test_split(
        X, y,lang, test_size=0.2, random_state=42
    )

    train_data = pd.DataFrame({'text': X_train, 'label': y_train})
    test_data = pd.DataFrame({'text': X_test, 'label': y_test})
    
    train_dataset = Dataset.from_pandas(train_data)
    test_dataset = Dataset.from_pandas(test_data)

    train_dataset = train_dataset.map(preprocess_function, batched=True)
    test_dataset = test_dataset.map(preprocess_function, batched=True)

    train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
    test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

    
    
    train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
    test_dataloader = DataLoader(test_dataset, batch_size=16,shuffle=True)
    model = XLMRobertaForSequenceClassification.from_pretrained("xlm-roberta-base", num_labels=22,problem_type="multi_label_classification").to(device)
    #tokenizer = XLMRobertaTokenizer.from_pretrained("xlm-roberta-base")
    optimizer = AdamW(model.parameters(), lr=4e-5)
    num_epochs = 8
    debug = 0

    del train_dataset 
    del test_dataset
    del train_data
    del test_data
    
    for epoch in range(num_epochs):
        # Training phase
        model.train()
        total_loss = 0
        correct_predictions = 0
        total_predictions = 0
        
        # Initialize tqdm progress bar for training
        train_progress_bar = tqdm(train_dataloader, desc=f"Training Epoch {epoch + 1}/{num_epochs}")
        
        for batch in train_progress_bar:
            optimizer.zero_grad()
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)
            
            # Forward pass
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels.float())
            loss = outputs.loss
            logits = outputs.logits
            
            # Backward pass and optimization
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
            
            # Calculate accuracy
            preds = (torch.sigmoid(logits) > 0.15).int()
            #print(preds)
            correct_predictions += ((preds == labels.int()).all(dim=1)).sum().item()
            total_predictions += labels.size(0)
            
            # Update tqdm description with current loss
            train_progress_bar.set_postfix({'loss': loss.item()})
        
        avg_train_loss = total_loss / len(train_dataloader)
        train_accuracy = correct_predictions / total_predictions
        
        print(f"Epoch {epoch + 1}/{num_epochs}")
        print(f"Training loss: {avg_train_loss:.4f}, Training accuracy: {train_accuracy:.4f}")
        
        # Test phase
        model.eval()
        test_loss = 0
        correct_test_predictions = 0
        total_test_predictions = 0
        
        # Initialize tqdm progress bar for test
        test_progress_bar = tqdm(test_dataloader, desc=f"Test Epoch {epoch + 1}/{num_epochs}")
        
        with torch.no_grad():
            for batch in test_progress_bar:
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                labels = batch['label'].to(device)
                        
                # Forward pass
                outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels.float())
                loss = outputs.loss
                logits = outputs.logits
                
                test_loss += loss.item()
                
                # Calculate test accuracy
                preds = (torch.sigmoid(logits) > 0.15).int()
                correct_test_predictions += ((preds == labels.int()).all(dim=1)).sum().item()
                total_test_predictions += labels.size(0)
                
                # Update tqdm description with current loss
                test_progress_bar.set_postfix({'loss': loss.item()})
        
        avg_test_loss = test_loss / len(test_dataloader)
        test_accuracy = correct_test_predictions / total_test_predictions
        
        print(f"Test loss: {avg_test_loss:.4f}, Test accuracy: {test_accuracy:.4f}")
        if epoch == num_epochs-1:
            accuracies_baseline.append(test_accuracy)
    del model
    del optimizer
    
    del X_train
    del X_test 
    #del y1_train 
    #del y1_test 
    del y_train 
    del y_test 
    del loss 
    del train_dataloader
    del test_dataloader
    torch.cuda.empty_cache()

Testing seed: 1


Map: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 702/702 [00:00<00:00, 1578.09 examples/s]
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Training Epoch 1/8: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [05:18<00:00,  1.81s/it, loss=0.222]


Epoch 1/8
Training loss: 0.2274, Training accuracy: 0.0521


Test Epoch 1/8: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [00:20<00:00,  2.12it/s, loss=0.148]


Test loss: 0.1788, Test accuracy: 0.1937


...

Test loss: 0.1758, Test accuracy: 0.1097


Training Epoch 8/8: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 176/176 [05:21<00:00,  1.83s/it, loss=0.169]


Epoch 8/8
Training loss: 0.1725, Training accuracy: 0.0317


Test Epoch 8/8: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [00:20<00:00,  2.12it/s, loss=0.146]


Test loss: 0.1758, Test accuracy: 0.0883


In [16]:
print(accuracies_baseline)

[0.0868945868945869, 0.09259259259259259, 0.07122507122507123, 0.06695156695156695, 0.05555555555555555, 0.1111111111111111, 0.08831908831908832]


In [17]:

# Open (or create) a file in write mode ('w')
with open("output2.txt", "w") as file:
    # Write each item in the list to the file
    for item in accuracies_baseline:
        file.write(f"{item}\n")  # Writing each item on a new line